# README 

Script rapide, très sale imo, mainly généré par IA pour voir vers quelle méthode aller.

GPU needed

TODO: 
* Comprendre l'éval du crossencodeur, j'ai pas pris le temps d'aller voir
* Better text preprocessing
* Test avec SciBERT !
* Train le bert jusqu'a early stop pour voir ces capacités
* Mesurer le temps d'inférence du BERT pour voir combien de temps ça nous prendrait

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from sentence_transformers import SentencesDataset, LoggingHandler, evaluation
from torch.utils.data import DataLoader
import torch
import numpy as np
import random

# Local GPU setup
from dotenv import load_dotenv
assert load_dotenv()
import os
assert os.getenv("HTTP_PROXY") is not None, "HTTP_PROXY must be set in .env file"
assert os.getenv('HF_HOME') is not None, "HF_HOME must be set in .env file"
# Set seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

/tempory/ehehboi/conda_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("data/full_concat_dataset.csv").dropna()

data = data.sample(frac=1, random_state=seed).reset_index(drop=True)
data["true_label"] = data["true_label"].replace({"Not About Sufficiency": 0, "About Sufficiency": 1})

X_train, X_test, y_train, y_test = train_test_split(
    data[["primary_title", "abstract"]],
    data["true_label"],
    test_size=0.2,
    stratify=data["true_label"],
    random_state=seed,
)


def preprocess_text(text):
    # TODO search for already existing preprocessing functions
    return text
X_train["abstract"] = X_train["abstract"].apply(preprocess_text)
X_test["abstract"] = X_test["abstract"].apply(preprocess_text)

assert not X_train["abstract"].isna().any()

/tmp/ipykernel_95213/3970361721.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["true_label"] = data["true_label"].replace({"Not About Sufficiency": 0, "About Sufficiency": 1})


In [3]:
# Traditional tfidf 
def train_traditional_model(X_train, y_train, X_test, y_test):
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train["abstract"])
    X_test_tfidf = vectorizer.transform(X_test["abstract"])

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_tfidf, y_train)

    y_pred = model.predict(X_test_tfidf)
    print("Logistic Regression Classification Report:")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
# Train and evaluate the models
print("Training Traditional Model...")
train_traditional_model(X_train, y_train, X_test, y_test)

Training Traditional Model...
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.52      0.61       394
           1       0.78      0.89      0.83       753

    accuracy                           0.77      1147
   macro avg       0.75      0.71      0.72      1147
weighted avg       0.76      0.77      0.76      1147

[[206 188]
 [ 80 673]]


In [4]:

# Transformer-based Model (BERT)
def train_bert_model(X_train, y_train, X_test, y_test):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    train_encodings = tokenizer(
        X_train["abstract"].tolist(), truncation=True, padding=True, max_length=512
    )
    test_encodings = tokenizer(
        X_test["abstract"].tolist(), truncation=True, padding=True, max_length=512
    )

    class AbstractDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = AbstractDataset(train_encodings, y_train.tolist())
    test_dataset = AbstractDataset(test_encodings, y_test.tolist())

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset
    )

    trainer.train()
    predictions = trainer.predict(test_dataset).predictions
    y_pred = np.argmax(predictions, axis=1)
    print("BERT Classification Report:")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    return (y_pred, y_test)


print("Training BERT Model...")
y_pred_bert, y_test_bert = train_bert_model(X_train, y_train, X_test, y_test)

Training BERT Model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.571800
1000,0.455300
1500,0.317800


BERT Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.65      0.71       394
           1       0.83      0.91      0.87       753

    accuracy                           0.82      1147
   macro avg       0.81      0.78      0.79      1147
weighted avg       0.82      0.82      0.81      1147

[[255 139]
 [ 68 685]]


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test_bert, y_pred_bert)).plot()

In [ ]:

# Cross-Encoder Model
def train_cross_encoder_model(X_train, y_train, X_test, y_test):
    model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

    train_examples = [
        InputExample(texts=[row["primary_title"], row["abstract"]], label=label)
        for row, label in zip(X_train.to_dict("records"), y_train)
    ]
    test_examples = [
        InputExample(texts=[row["primary_title"], row["abstract"]], label=label)
        for row, label in zip(X_test.to_dict("records"), y_test)
    ]

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    train_loss = losses.CosineSimilarityLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)

    test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=16)
    evaluator = evaluation.BinaryClassificationEvaluator.from_input_examples(
        test_examples, name="sufficiency-eval"
    )

    result = model.evaluate(evaluator)
    print(result)
    return result


print("Training Cross-Encoder Model...")
y_result_cross_encoder = train_cross_encoder_model(X_train, y_train, X_test, y_test)


Training Cross-Encoder Model...


Step,Training Loss
500,0.177700


{'sufficiency-eval_cosine_accuracy': 0.7646033129904097, 'sufficiency-eval_cosine_accuracy_threshold': 0.5688587427139282, 'sufficiency-eval_cosine_f1': 0.824126984126984, 'sufficiency-eval_cosine_f1_threshold': 0.47997307777404785, 'sufficiency-eval_cosine_precision': 0.7895377128953771, 'sufficiency-eval_cosine_recall': 0.8618857901726428, 'sufficiency-eval_cosine_ap': 0.889762105477786, 'sufficiency-eval_cosine_mcc': 0.44555617855751745}
